## Démonstration des techniques de découpage de texte avec Python

Dans ce notebook, nous explorons plusieurs techniques de découpage de texte, utiles dans le cadre de tâches de NLP (traitement du langage naturel), de résumé, ou d'indexation pour des agents conversationnels.

Nous allons couvrir :

- Le découpage récursif avec chevauchement (LangChain)

- Le découpage basé sur les balises (HTML/Markdown)

- Le découpage sémantique avec un modèle NLP (spaCy)

In [ ]:
!pip install langchain --quiet

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialisation du splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Taille de chaque segment
    chunk_overlap=200     # Chevauchement entre les segments
)

# Exemple d'utilisation
texte = """Votre texte ici. Ceci est un exemple illustrant la manière dont le texte est découpé en morceaux..."""
segments = text_splitter.split_text(texte)

for i, segment in enumerate(segments):
    print(f"Segment {i+1}:\n{segment}\n")

In [ ]:
!pip install beautifulsoup4 --quiet

from bs4 import BeautifulSoup

def chunk_by_tags(html_content, tags):
    soup = BeautifulSoup(html_content, 'html.parser')
    segments = []
    for tag in tags:
        elements = soup.find_all(tag)
        for elem in elements:
            segments.append(elem.get_text())
    return segments

# Exemple
html_content = "<h1>Titre</h1><p>Paragraphe 1</p><p>Paragraphe 2</p>"
tags = ['h1', 'p']
segments = chunk_by_tags(html_content, tags)

for i, segment in enumerate(segments):
    print(f"Segment {i+1}:\n{segment}\n")

In [ ]:
!pip install spacy --quiet
!python -m spacy download fr_core_news_sm

import spacy

# Charger le modèle français
nlp = spacy.load("fr_core_news_sm")

def semantic_chunking(text, max_chunk_size=150):
    doc = nlp(text)
    segments = []
    current_segment = []

    for sent in doc.sents:
        if len(" ".join(current_segment)) + len(sent.text) <= max_chunk_size:
            current_segment.append(sent.text)
        else:
            segments.append(" ".join(current_segment))
            current_segment = [sent.text]
    if current_segment:
        segments.append(" ".join(current_segment))
    return segments

# Exemple
texte = "Ceci est une première phrase. Voici une autre phrase importante. Encore une phrase pour tester."
segments = semantic_chunking(texte)

for i, segment in enumerate(segments):
    print(f"Segment {i+1}:\n{segment}\n")